In [1]:
# imports
import pandas as pd
import numpy as np
import re
import nltk
from tqdm.notebook import tqdm
from IPython.core.interactiveshell import InteractiveShell
from nltk.corpus import stopwords

In [2]:
# import configurations
InteractiveShell.ast_node_interactivity = "all"
tqdm.pandas()
nltk.download('stopwords')
english_stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kevctae/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# read file
df = pd.read_pickle("../data/interim/tweets_verified_2020-2021.pkl")
df.head()

created_at                   id      conversation_id  \
0 2020-01-01 00:08:28-05:00  1212239143687741440  1212239143687741440   
1 2020-01-01 00:54:35-05:00  1212250748815208448  1212250748815208448   
2 2020-01-01 02:00:25-05:00  1212267316789952512  1212267316789952512   
3 2020-01-01 02:18:48-05:00  1212271940502638593  1212271940502638593   
4 2020-01-01 02:39:29-05:00  1212277146401402880  1212277146401402880   

               user_id        username              name  \
0             20646945    dumbfoundead  dumbfoundead.eth   
1            636023721    neuroecology    Adam J Calhoun   
2            631810714    cnbctv18news         CNBC-TV18   
3             14654494       excellion        Samson Mow   
4  1066972567943053312  hindustantimes   Hindustan Times   

                                               tweet language  \
0      Bitcoin the worst decision i made this decade       en   
1  If you ask someone what they'd do if they went...       en   
2  #bitcoin rally begun in 2013 and it reached a ...       en   
3  Wishing all #Bitcoin Ultra Enthusiasts a very ...       en   
4  Breaking down Bitcoin’s 9,000,000% rise in las...       en   

                                            mentions  \
0                                                 []   
1                                                 []   
2                                                 []   
3  [{'screen_name': 'blockstream', 'name': 'block...   
4                                                 []   

                                                urls  ... replies_count  \
0                                                 []  ...             9   
1                                                 []  ...             6   
2  ['https://www.cnbctv18.com/market/currency/bit...  ...             0   
3                                                 []  ...            10   
4  ['http://www.hindustantimes.com/tech/breaking-...  ...             0   

   retweets_count  likes_count                      hashtags cashtags  \
0              15          409                            []       []   
1               0            4                            []       []   
2               0            4                   ['bitcoin']       []   
3              17          158  ['bitcoin', 'liquidnetwork']       []   
4               0            0                            []       []   

                                                link quote_url  video  \
0  https://twitter.com/dumbfoundead/status/121223...      <NA>  False   
1  https://twitter.com/neuroecology/status/121225...      <NA>  False   
2  https://twitter.com/CNBCTV18News/status/121226...      <NA>  False   
3  https://twitter.com/Excellion/status/121227194...      <NA>   True   
4  https://twitter.com/HindustanTimes/status/1212...      <NA>  False   

                                         thumbnail reply_to  
0                                             <NA>       []  
1                                             <NA>       []  
2                                             <NA>       []  
3  https://pbs.twimg.com/media/ENLa5toU0AAvhC1.jpg       []  
4                                             <NA>       []  

[5 rows x 21 columns]

In [4]:
def clean_tweet_1(tweet):
    tweet = tweet.lower()
    # remove URL "https://t.co/"
    new_url = re.sub(r"https://t.co/[A-Za-z0-9]+", " ", tweet)
    
    # remove mention
    new_mention = re.sub(r"@[A-Za-z0-9!#%&*;_\$\.]+", " ", new_url)
    
    # remove # ,turning hashtags to the typical words.
    new_symbol = re.sub(r"\W+", " ", tweet)
    
    return new_symbol

In [5]:
# language EN only
df = df[df['language'] == 'en']

# apply cleaninging 1
df['tweet'] = df['tweet'].progress_apply(clean_tweet_1)

  0%|          | 0/312204 [00:00<?, ?it/s]

In [6]:
def repl(matchObj):
    char = matchObj.group(1)
    return "%s%s" % (char, char)

def clean_tweet_2(tweet):
    # reduce character sequences >3 to 3
    new_sequence = re.sub(re.compile(r"(\w)\1+"), repl, tweet)

    # remove 2-character words
    new_twochar = re.sub(r"\b[a-z]{1,2}\b", " ", new_sequence)
    
    # replace 2 or more spaces with a single space.
    new_space = re.sub(r"\s+", " ", new_twochar)
    
    # remove spaces front and back
    new_space_end = re.sub(r"^\s+|\s$", "", new_space)
    
    return new_space_end

# remove stopword
def remove_stop_words(tweet):
    
    tweet_removed = ' '.join([word for word in tweet.split() 
                      if word not in english_stop_words])
    
    return tweet_removed

In [7]:
df['tweet'] = df['tweet'].progress_apply(remove_stop_words)
df['tweet'] = df['tweet'].progress_apply(clean_tweet_2)

  0%|          | 0/312204 [00:00<?, ?it/s]

  0%|          | 0/312204 [00:00<?, ?it/s]

In [8]:
df.head()

created_at                   id      conversation_id  \
0 2020-01-01 00:08:28-05:00  1212239143687741440  1212239143687741440   
1 2020-01-01 00:54:35-05:00  1212250748815208448  1212250748815208448   
2 2020-01-01 02:00:25-05:00  1212267316789952512  1212267316789952512   
3 2020-01-01 02:18:48-05:00  1212271940502638593  1212271940502638593   
4 2020-01-01 02:39:29-05:00  1212277146401402880  1212277146401402880   

               user_id        username              name  \
0             20646945    dumbfoundead  dumbfoundead.eth   
1            636023721    neuroecology    Adam J Calhoun   
2            631810714    cnbctv18news         CNBC-TV18   
3             14654494       excellion        Samson Mow   
4  1066972567943053312  hindustantimes   Hindustan Times   

                                               tweet language  \
0                 bitcoin worst decision made decade       en   
1  ask someone went back ten years use informatio...       en   
2  bitcoin rally begun 2013 reached peak 20 00 20...       en   
3  wishing bitcoin ultra enthusiasts happy new ye...       en   
4  breaking bitcoin 9 00 00 rise last decade left...       en   

                                            mentions  \
0                                                 []   
1                                                 []   
2                                                 []   
3  [{'screen_name': 'blockstream', 'name': 'block...   
4                                                 []   

                                                urls  ... replies_count  \
0                                                 []  ...             9   
1                                                 []  ...             6   
2  ['https://www.cnbctv18.com/market/currency/bit...  ...             0   
3                                                 []  ...            10   
4  ['http://www.hindustantimes.com/tech/breaking-...  ...             0   

   retweets_count  likes_count                      hashtags cashtags  \
0              15          409                            []       []   
1               0            4                            []       []   
2               0            4                   ['bitcoin']       []   
3              17          158  ['bitcoin', 'liquidnetwork']       []   
4               0            0                            []       []   

                                                link quote_url  video  \
0  https://twitter.com/dumbfoundead/status/121223...      <NA>  False   
1  https://twitter.com/neuroecology/status/121225...      <NA>  False   
2  https://twitter.com/CNBCTV18News/status/121226...      <NA>  False   
3  https://twitter.com/Excellion/status/121227194...      <NA>   True   
4  https://twitter.com/HindustanTimes/status/1212...      <NA>  False   

                                         thumbnail reply_to  
0                                             <NA>       []  
1                                             <NA>       []  
2                                             <NA>       []  
3  https://pbs.twimg.com/media/ENLa5toU0AAvhC1.jpg       []  
4                                             <NA>       []  

[5 rows x 21 columns]

In [11]:
# export as pickle
df = df.astype({'tweet':'string'}).reset_index(drop=True)
df.to_pickle("../data/cleaned/tweets_verified_2020-2021_cleaned.pkl")

In [12]:
# try reading pickle
df_test = pd.read_pickle("../data/cleaned/tweets_verified_2020-2021_cleaned.pkl")

display(df_test.dtypes)
display(df_test)

created_at         datetime64[ns, tzlocal()]
id                                     int64
conversation_id                        int64
user_id                                int64
username                              string
name                                  string
tweet                                 string
language                              string
mentions                              object
urls                                  object
photos                                object
replies_count                          int64
retweets_count                         int64
likes_count                            int64
hashtags                              object
cashtags                              object
link                                  string
quote_url                             string
video                                   bool
thumbnail                             string
reply_to                              object
dtype: object

created_at                   id      conversation_id  \
0      2020-01-01 00:08:28-05:00  1212239143687741440  1212239143687741440   
1      2020-01-01 00:54:35-05:00  1212250748815208448  1212250748815208448   
2      2020-01-01 02:00:25-05:00  1212267316789952512  1212267316789952512   
3      2020-01-01 02:18:48-05:00  1212271940502638593  1212271940502638593   
4      2020-01-01 02:39:29-05:00  1212277146401402880  1212277146401402880   
...                          ...                  ...                  ...   
312199 2021-12-31 23:20:12-05:00  1477132523826147330  1477131257301520385   
312200 2021-12-31 23:28:45-05:00  1477134677148323841  1477134677148323841   
312201 2021-12-31 23:38:06-05:00  1477137028894670855  1477132718341337095   
312202 2021-12-31 23:43:13-05:00  1477138318982725633  1477138318982725633   
312203 2021-12-31 23:55:44-05:00  1477141468921794560  1477141468921794560   

                    user_id        username                    name  \
0                  20646945    dumbfoundead        dumbfoundead.eth   
1                 636023721    neuroecology          Adam J Calhoun   
2                 631810714    cnbctv18news               CNBC-TV18   
3                  14654494       excellion              Samson Mow   
4       1066972567943053312  hindustantimes         Hindustan Times   
...                     ...             ...                     ...   
312199            151805556      adeldmeyer                    Adel   
312200              1414971        jaygould               Jay Gould   
312201             10446482             nvk  DETERMINISTIC OPTIMISM   
312202             22594051      staronline                The Star   
312203             36624001     lordfusitua           Lord Fusitu'a   

                                                    tweet language  \
0                      bitcoin worst decision made decade       en   
1       ask someone went back ten years use informatio...       en   
2       bitcoin rally begun 2013 reached peak 20 00 20...       en   
3       wishing bitcoin ultra enthusiasts happy new ye...       en   
4       breaking bitcoin 9 00 00 rise last decade left...       en   
...                                                   ...      ...   
312199                           bitcoin__art fair enough       en   
312200            happy new year bitcoin https hduqndkuua       en   
312201  rubiconcapital_ buy bitcoin amp chill defund p...       en   
312202  bitcoin faces uncertain 2022 record year https...       en   
312203       great example fellow bitcoin familia brother       en   

                                                 mentions  \
0                                                      []   
1                                                      []   
2                                                      []   
3       [{'screen_name': 'blockstream', 'name': 'block...   
4                                                      []   
...                                                   ...   
312199                                                 []   
312200                                                 []   
312201                                                 []   
312202                                                 []   
312203                                                 []   

                                                     urls  ... replies_count  \
0                                                      []  ...             9   
1                                                      []  ...             6   
2       ['https://www.cnbctv18.com/market/currency/bit...  ...             0   
3                                                      []  ...            10   
4       ['http://www.hindustantimes.com/tech/breaking-...  ...             0   
...                                                   ...  ...           ...   
312199                                                 []  ...             0   
312200            